In [1]:
import numpy as np
import os
import sys
import copy
import math
import csv

In [2]:
dataset_names = ['gtea', '50salads', 'breakfast']
backbone_names = ['asrf', 'mstcn', 'sstda', 'mgru']
num_splits = dict()
num_splits['gtea'] = 4
num_splits['50salads']=5
num_splits['breakfast']=4

In [3]:
record_root = './record'
refiner_best_epoch = dict()
backbone_best_epoch = dict()

for dir_name in sorted([x for x in os.listdir(record_root) if x[0]!='.']):
    backbone_name = ''.join([t for t in dir_name if t.isupper()]).lower()
    if len(backbone_name) > 0:
        refiner_best_epoch[dir_name] = {dn:[] for dn in dataset_names}
        
        backbone_best_epoch[backbone_name] = {dn:[] for dn in dataset_names}
        
        for data_name in os.listdir(os.path.join(record_root, dir_name)):
            print(backbone_name, '/\t', dir_name, '/\t', data_name)
            csv_list = os.listdir(os.path.join(record_root, dir_name, data_name))
            plot_flag = True
            for i in range(num_splits[data_name]):
                if 'split_{}_best.csv'.format(i+1) not in csv_list:
                    plot_flag = False
            if plot_flag:
                curr_score = np.asarray([0.0 for _ in range(5)])
                backbone_score = np.asarray([0.0 for _ in range(5)])
                for i in range(num_splits[data_name]):
                    curr_csv_fp = os.path.join(record_root, dir_name, data_name, 'split_{}_best.csv'.format(i+1))
                    backbone_csv_fp = os.path.join(record_root, backbone_name, data_name, 'split_{}_best.csv'.format(i+1))
                    with open(curr_csv_fp, 'r') as f:
                        reader = csv.reader(f, delimiter='\t')
                        for ri, row in enumerate(reader):
                            if ri>0:
                                refiner_best_epoch[dir_name][data_name].append(int(row[0]))
                                curr_score += np.asarray([float(r) for r in row[1:]]) / num_splits[data_name]
                    with open(backbone_csv_fp, 'r') as f:
                        reader = csv.reader(f, delimiter='\t')
                        for ri, row in enumerate(reader):
                            if ri>0:
                                backbone_best_epoch[backbone_name][data_name].append(int(row[0]))
                                backbone_score += np.asarray([float(r) for r in row[1:]]) / num_splits[data_name]
                print('backbone:\t %.1f / %.1f / %.1f / %.1f /%.1f ' %(backbone_score[2], backbone_score[3], backbone_score[4], backbone_score[1], backbone_score[0]))
                print('refined:\t %.1f / %.1f / %.1f / %.1f /%.1f ' %(curr_score[2], curr_score[3], curr_score[4], curr_score[1], curr_score[0]))
                print('gain: \t\t %.1f / %.1f / %.1f / %.1f /%.1f ' %( curr_score[2] - backbone_score[2],
                                                                      curr_score[3]-backbone_score[3],
                                                                      curr_score[4]-backbone_score[4],
                                                                      curr_score[1]-backbone_score[1],
                                                                      curr_score[0]-backbone_score[0])) 
            print('-'*80)



asrf /	 refinerASRF-asrf /	 breakfast
backbone:	 73.8 / 68.6 / 56.4 / 72.2 /68.5 
refined:	 74.7 / 69.5 / 57.0 / 71.9 /69.4 
gain: 		 0.9 / 1.0 / 0.7 / -0.3 /0.9 
--------------------------------------------------------------------------------
asrf /	 refinerASRF-asrf /	 gtea
backbone:	 87.9 / 86.1 / 75.2 / 81.9 /77.1 
refined:	 88.9 / 87.4 / 75.6 / 83.8 /77.7 
gain: 		 1.0 / 1.3 / 0.4 / 1.9 /0.6 
--------------------------------------------------------------------------------
asrf /	 refinerASRF-asrf /	 50salads
backbone:	 85.1 / 83.3 / 77.7 / 79.9 /83.7 
refined:	 86.5 / 85.4 / 78.5 / 81.0 /83.9 
gain: 		 1.3 / 2.1 / 0.8 / 1.1 /0.2 
--------------------------------------------------------------------------------
asrf /	 refinerASRF-sstda-mstcn-mgru /	 gtea
backbone:	 87.9 / 86.1 / 75.2 / 81.9 /77.1 
refined:	 88.2 / 85.9 / 73.9 / 81.2 /76.9 
gain: 		 0.3 / -0.1 / -1.3 / -0.6 /-0.2 
--------------------------------------------------------------------------------
asrf /	 refinerASRF-ss

#### 